<a href="https://colab.research.google.com/github/hdakhli/mlops-2024/blob/main/21_convolutional_neural_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification en utilisant un réseau de neurones convolutif (CNN)

In [1]:
!pip install tensorflow

In [2]:
from keras import models
from keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

2024-05-27 14:25:09.097277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Exemple d'architecture d'un réseau CNN:
# Nous initialisons un modèle vide, nous commençons ensuite à le construire couche par couche. Ici, nous avons une couche de convolution avec plusieurs paramètres :
# 
# 32 filtres
# un motif de taille (3, 3), aussi appelé kernel 3×3 pixels
# un pas de (1,1), aussi appelé strides
# une fonction d’activation.
# taille des données d'entrée (28, 28, 1) une image en 3 dimensions (hauteur, largeur, couleur), un 3D-tensor.
# Cette couche passe sur chaque pixel de l’image (strides = (1,1)) pour en extraire des motifs de taille 3×3 pixels (kernel = (3,3)). La couche exécute cette action 32 fois (filtres = 32) et produit 32 feature-maps chacune représente des caractéristiques différentes de l’image.

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform',input_shape=(28, 28, 1)))


# La couche suivante est une couche de MaxPooling. Il existe deux principaux types de couches de regroupement :
# 
# Max Pooling : lorsque le filtre se déplace sur l’entrée, il sélectionne le pixel avec la valeur maximale à envoyer à la matrice de sortie. Cette approche tend à être utilisée plus souvent que la mise en commun moyenne.
# Average Pooling : Lorsque le filtre se déplace sur l’entrée, il calcule la valeur moyenne dans le champ réceptif à envoyer à la matrice de sortie.
# Nous allons utiliser une couche MaxPooling a un kernel de taille (2, 2) dans notre cas.

model.add(MaxPooling2D((2, 2)))


# Puisque nous avons commencé avec une image en 3 dimensions (hauteur, largeur, couleur), un 3D-tensor, et nous finissons avec un label à une dimension, un 1D-tensor. Une couche de convolution retourne un 3D-tensor, un tenseur à 3 dimensions, ce ne peut donc pas être la couche finale. Nous utilisons alors une couche appelée Flatten qui permet d’aplatir le tenseur, de réduire sa dimension. Elle prend en entrée un 3D-tensor et retourne un 1D-tensor.
model.add(Flatten())


# Dans notre cas, nous ajoutons deux couches entièrement connectées, la couche finale produit en sortie un vecteur de taille 10. 10 fait référence au nombre de classes de digits (de 0 à 9).
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))